In [ ]:
from timing_analysis.lite_utils import *
from timing_analysis.par_checker import *
from timing_analysis.utils import *
from timing_analysis.dmx_utils import *
from timing_analysis.timingconfiguration import TimingConfiguration
from timing_analysis.outlier_utils import epochalyptica, calculate_pout
from timing_analysis.utils import apply_cut_flag, apply_cut_select
import yaml
from astropy import log
import pint.fitter
from pint.utils import dmxparse
import os

In [ ]:
config = "configs/[psr_name].[nb or wb].yaml"  # fill in actual path
par_directory = None   # default location
tim_directory = None   # default location
tc = TimingConfiguration(config, par_directory=par_directory, tim_directory=tim_directory)

using_wideband = tc.get_toa_type() == 'WB'
mo,to = tc.get_model_and_toas(apply_initial_cuts=True)
to = setup_dmx(mo,to,frequency_ratio=tc.get_fratio(),max_delta_t=tc.get_sw_delay())

In [ ]:
run_outlier_analysis = True
use_outlier_results = False

if run_outlier_analysis:
    calculate_pout(mo, to, file_base=tc.get_outfile_basename(), results_dir='outlier', method='gibbs', Nsamples=10000)

if use_outlier_results:
    pass
    # Deal with this later

In [ ]:
# Re-introduce cut TOAs for writing tim files that include -cut/-pout flags
to.table = to.orig_table
fo = tc.construct_fitter(to,mo)
write_tim(fo,toatype=tc.get_toa_type(),outfile=f'{tc.get_outfile_basename()}_out.tim')